In [1]:
from tensorflow import keras
import tensorflow as tf
#tf.debugging.set_log_device_placement(True)
# Set CPU as available physical device
#my_devices = tf.config.experimental.list_physical_devices(device_type='CPU')
#tf.config.experimental.set_visible_devices(devices= my_devices, device_type='CPU')
import tensorflow_datasets as tfds
import numpy as np
from datetime import datetime

device_name = tf.test.gpu_device_name()
#if not device_name:
#    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
def custom_l1_reg(weight_matrix):
    return 0.01 * tf.math.reduce_sum(tf.math.abs(weight_matrix))

def custom_loss_func(y_actual, y_predicted):
    custom_loss_value=tf.math.reduce_mean(tf.math.square(y_actual-y_predicted))
    return custom_loss_value

# SGD

In [3]:
tf.keras.backend.clear_session()
keras_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(272*2, activation=tf.nn.relu, input_shape=[1], use_bias=False,
                      kernel_regularizer=custom_l1_reg),
    keras.layers.Dense(1, use_bias=False)
])

In [4]:
optimizer = tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.)
keras_model.compile(loss=custom_loss_func, optimizer=optimizer)

In [5]:
ds = tfds.load('mnist', split='train', shuffle_files=True)

In [6]:
# Build your input pipeline
batch=1
ds = ds.shuffle(1024).batch(batch).prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
%%timeit

for example in ds.take(100):
    image, label = example["image"], example["label"]
    keras_model.fit(image,tf.constant(np.zeros(shape=batch)),verbose=0)
                    #callbacks = [tboard_callback])

1.48 s ± 7.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# ADAM

In [8]:
tf.keras.backend.clear_session()
keras_model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(272*2, activation=tf.nn.relu, input_shape=[1], use_bias=False,
                      kernel_regularizer=custom_l1_reg),
    keras.layers.Dense(1, use_bias=False)
])

In [9]:
optimizer = tf.keras.optimizers.Adam()
keras_model.compile(loss=custom_loss_func, optimizer=optimizer)

In [10]:
ds = tfds.load('mnist', split='train', shuffle_files=True)

In [11]:
# Build your input pipeline
batch=1
ds = ds.shuffle(1024).batch(batch).prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
%%timeit

for example in ds.take(100):
    image, label = example["image"], example["label"]
    keras_model.fit(image,tf.constant(np.zeros(shape=batch)),verbose=0)
                    #callbacks = [tboard_callback])

1.49 s ± 3.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
keras_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  426496    
_________________________________________________________________
dense_1 (Dense)              multiple                  544       
Total params: 427,040
Trainable params: 427,040
Non-trainable params: 0
_________________________________________________________________
